In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import math
import random
import shutil
import argparse
import importlib
import data_utils_legacy as data_utils
import numpy as np
import pointfly as pf
import tensorflow as tf
from datetime import datetime

import h5py

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--filelist', '-t', default="data/KITTI/ImageSets/train.txt", help='Path to training set ground truth (.txt)')
parser.add_argument('--filelist_val', '-v', default="data/KITTI/ImageSets/train.txt", help='Path to validation set ground truth (.txt)')
parser.add_argument('--load_ckpt', '-l', help='Path to a check point file for load')
parser.add_argument('--save_folder', '-s', default="/home/kartik/saved/", help='Path to folder for saving check points and summary')
parser.add_argument('--model', '-m', default="pointcnn_seg", help='Model to use')
parser.add_argument('--setting', '-x', default="kitti3d_x8_2048_fps", help='Setting to use')
parser.add_argument('--epochs', default="1",help='Number of training epochs (default defined in setting)', type=int)
parser.add_argument('--batch_size', default="1", help='Batch size (default defined in setting)', type=int)
# default="64",
parser.add_argument('--log', help='Log to FILE in save folder; use - for stdout (default is log.txt)', metavar='FILE', default='log.txt')
parser.add_argument('--no_timestamp_folder', help='Dont save to timestamp folder', action='store_true')
parser.add_argument('--no_code_backup', help='Dont backup code', action='store_true')
args = parser.parse_args([])

print(args)

Namespace(batch_size=1, epochs=1, filelist='data/KITTI/ImageSets/train.txt', filelist_val='data/KITTI/ImageSets/train.txt', load_ckpt=None, log='log.txt', model='pointcnn_seg', no_code_backup=False, no_timestamp_folder=False, save_folder='/home/kartik/saved/', setting='kitti3d_x8_2048_fps')


In [3]:
cwd = os.getcwd()
model = importlib.import_module(args.model)
setting_path = os.path.join(cwd, args.model)
sys.path.append(setting_path)
setting = importlib.import_module(args.setting)

num_epochs = args.epochs or setting.num_epochs
batch_size = args.batch_size or setting.batch_size
sample_num = setting.sample_num
step_val = setting.step_val
label_weights_list = setting.label_weights
rotation_range = setting.rotation_range
rotation_range_val = setting.rotation_range_val
scaling_range = setting.scaling_range
scaling_range_val = setting.scaling_range_val
jitter = setting.jitter
jitter_val = setting.jitter_val

In [4]:
points = []
labels = []
point_nums = []
labels_seg = []
indices_split_to_full = []

print('{}-Preparing datasets...'.format(datetime.now()))
# is_list_of_h5_list = data_utils.is_h5_list(args.filelist)
args.filelist = os.path.join(cwd, "data", "Argo_h5_noGR_scaled_depth", "train")+"/train.txt"
args.filelist_val = os.path.join(cwd, "data", "Argo_h5_noGR_scaled_depth", "train")+"/val.txt"

is_list_of_h5_list = not data_utils.is_h5_list(args.filelist)
if is_list_of_h5_list:
    seg_list = data_utils.load_seg_list(args.filelist)
    seg_list_idx = 0
    filelist_train = seg_list[seg_list_idx]
    seg_list_idx = seg_list_idx + 1
else:
    filelist_train = args.filelist
data_train, _, data_num_train, label_train, _ = data_utils.load_seg_Argo(filelist_train)
data_val, _, data_num_val, label_val, _ = data_utils.load_seg_Argo(args.filelist_val)


data_train = np.dstack([data_train, label_train[:, :,np.newaxis]])
data_val = np.dstack([data_val, label_val[:, :,np.newaxis]])

# shuffle
data_train, data_num_train, label_train = \
    data_utils.grouped_shuffle([data_train, data_num_train, label_train])

2019-12-30 13:48:55.489688-Preparing datasets...


FileNotFoundError: [Errno 2] No such file or directory: '/home/kartik/DL_model/PointCNN/data/Argo_h5_noGR_scaled_depth/train/train.txt'

In [5]:
num_train = data_train.shape[0]
point_num = data_train.shape[1]
num_val = data_val.shape[0]
print('{}-{:d}/{:d} training/validation samples.'.format(datetime.now(), num_train, num_val))
batch_num = (num_train * num_epochs + batch_size - 1) // batch_size
print('{}-{:d} training batches.'.format(datetime.now(), batch_num))
batch_num_val = math.ceil(num_val / batch_size)
print('{}-{:d} testing batches per test.'.format(datetime.now(), batch_num_val))

2019-12-30 13:04:41.132878-43942/1639 training/validation samples.
2019-12-30 13:04:41.133007-43942 training batches.
2019-12-30 13:04:41.133113-1639 testing batches per test.


In [6]:
######################################################################
# Placeholders
indices = tf.placeholder(tf.int32, shape=(None, None, 2), name="indices")
xforms = tf.placeholder(tf.float32, shape=(None, 3, 3), name="xforms")
rotations = tf.placeholder(tf.float32, shape=(None, 3, 3), name="rotations")
jitter_range = tf.placeholder(tf.float32, shape=(1), name="jitter_range")
global_step = tf.Variable(0, trainable=False, name='global_step')
is_training = tf.placeholder(tf.bool, name='is_training')

pts_fts = tf.placeholder(tf.float32, shape=(None, point_num, setting.data_dim), name='pts_fts')
labels_seg = tf.placeholder(tf.int64, shape=(None, point_num), name='labels_seg')
labels_weights = tf.placeholder(tf.float32, shape=(None, point_num), name='labels_weights')

######################################################################
pts_fts_sampled = tf.gather_nd(pts_fts, indices=indices, name='pts_fts_sampled')
features_augmented = None

In [7]:
if setting.data_dim > 3:
    points_sampled, features_sampled = tf.split(pts_fts_sampled,
                                                [3, setting.data_dim - 3],
                                                axis=-1,
                                                name='split_points_features')
    if setting.use_extra_features:
        if setting.with_normal_feature:
            if setting.data_dim < 6:
                print('Only 3D normals are supported!')
                exit()
            elif setting.data_dim == 6:
                features_augmented = pf.augment(features_sampled, rotations)
            else:
                normals, rest = tf.split(features_sampled, [3, setting.data_dim - 6])
                normals_augmented = pf.augment(normals, rotations)
                features_augmented = tf.concat([normals_augmented, rest], axis=-1)
        else:
            features_augmented = features_sampled
else:
    points_sampled = pts_fts_sampled
points_augmented = pf.augment(points_sampled, xforms, jitter_range)


In [8]:
labels_sampled = tf.gather_nd(labels_seg, indices=indices, name='labels_sampled')
labels_weights_sampled = tf.gather_nd(labels_weights, indices=indices, name='labels_weight_sampled')

net = model.Net(points_augmented, features_augmented, is_training, setting)
logits = net.logits
probs = tf.nn.softmax(logits, name='probs')
predictions = tf.argmax(probs, axis=-1, name='predictions')

loss_op = tf.losses.sparse_softmax_cross_entropy(labels=labels_sampled, logits=logits,
                                                 weights=labels_weights_sampled)

with tf.name_scope('metrics'):
    loss_mean_op, loss_mean_update_op = tf.metrics.mean(loss_op)
    t_1_acc_op, t_1_acc_update_op = tf.metrics.accuracy(labels_sampled, predictions, weights=labels_weights_sampled)
    t_1_per_class_acc_op, t_1_per_class_acc_update_op = \
        tf.metrics.mean_per_class_accuracy(labels_sampled, predictions, setting.num_class,
                                           weights=labels_weights_sampled)
reset_metrics_op = tf.variables_initializer([var for var in tf.local_variables()
                                             if var.name.split('/')[0] == 'metrics'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
_ = tf.summary.scalar('loss/train', tensor=loss_mean_op, collections=['train'])
_ = tf.summary.scalar('t_1_acc/train', tensor=t_1_acc_op, collections=['train'])
_ = tf.summary.scalar('t_1_per_class_acc/train', tensor=t_1_per_class_acc_op, collections=['train'])

_ = tf.summary.scalar('loss/val', tensor=loss_mean_op, collections=['val'])
_ = tf.summary.scalar('t_1_acc/val', tensor=t_1_acc_op, collections=['val'])
_ = tf.summary.scalar('t_1_per_class_acc/val', tensor=t_1_per_class_acc_op, collections=['val'])

lr_exp_op = tf.train.exponential_decay(setting.learning_rate_base, global_step, setting.decay_steps,
                                       setting.decay_rate, staircase=True)
lr_clip_op = tf.maximum(lr_exp_op, setting.learning_rate_min)
_ = tf.summary.scalar('learning_rate', tensor=lr_clip_op, collections=['train'])
reg_loss = setting.weight_decay * tf.losses.get_regularization_loss()
if setting.optimizer == 'adam':
    optimizer = tf.train.AdamOptimizer(learning_rate=lr_clip_op, epsilon=setting.epsilon)
elif setting.optimizer == 'momentum':
    optimizer = tf.train.MomentumOptimizer(learning_rate=lr_clip_op, momentum=setting.momentum, use_nesterov=True)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = optimizer.minimize(loss_op + reg_loss, global_step=global_step)

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

saver = tf.train.Saver(max_to_keep=None)


In [11]:
# backup all code
# if not args.no_code_backup:
#     code_folder = os.path.abspath(os.path.dirname(__file__))
#     shutil.copytree(code_folder, os.path.join(root_folder, os.path.basename(code_folder)))

root_folder = os.path.join(cwd, "outfolder_argo_gr")
if not os.path.exists(root_folder):
    os.makedirs(root_folder)
# shutil.copytree(code_folder, os.path.join(root_folder, os.path.basename(code_folder)))

folder_ckpt = os.path.join(root_folder, 'ckpts')
if not os.path.exists(folder_ckpt):
    os.makedirs(folder_ckpt)

folder_summary = os.path.join(root_folder, 'summary')
if not os.path.exists(folder_summary):
    os.makedirs(folder_summary)

parameter_num = np.sum([np.prod(v.shape.as_list()) for v in tf.trainable_variables()])
print('{}-Parameter number: {:d}.'.format(datetime.now(), parameter_num))


2019-12-30 13:10:17.538809-Parameter number: 11513360.


In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)

with tf.Session( config=tf.ConfigProto(gpu_options=gpu_options) ) as sess:
    summaries_op = tf.summary.merge_all('train')
    summaries_val_op = tf.summary.merge_all('val')
    summary_writer = tf.summary.FileWriter(folder_summary, sess.graph)

    sess.run(init_op)

    # Load the model
    if args.load_ckpt is not None:
        saver.restore(sess, args.load_ckpt)
        print('{}-Checkpoint loaded from {}!'.format(datetime.now(), args.load_ckpt))
    else:
        latest_ckpt = tf.train.latest_checkpoint(folder_ckpt)
        if latest_ckpt:
            print('{}-Found checkpoint {}'.format(datetime.now(), latest_ckpt))
            saver.restore(sess, latest_ckpt)
            print('{}-Checkpoint loaded from {} (Iter {})'.format(
                datetime.now(), latest_ckpt, sess.run(global_step)))

    for batch_idx_train in range(batch_num):


        ######################################################################
        # Training
        start_idx = (batch_size * batch_idx_train) % num_train
        end_idx = min(start_idx + batch_size, num_train)
        batch_size_train = end_idx - start_idx

        points_batch = data_train[start_idx:end_idx, ...] # data number
        points_num_batch = data_num_train[start_idx:end_idx, ...] # Number of data points 
        labels_batch = label_train[start_idx:end_idx, ...] # labels of segmentation for each point
        weights_batch = np.array(label_weights_list)[labels_batch] 


        if start_idx + batch_size_train == num_train:
            if is_list_of_h5_list:
                filelist_train_prev = seg_list[(seg_list_idx - 1) % len(seg_list)]
                filelist_train = seg_list[seg_list_idx % len(seg_list)]
                if filelist_train != filelist_train_prev:
                    # Load the train data and labels 
                    data_train, _, data_num_train, label_train, _ = data_utils.load_seg(filelist_train)
                    num_train = data_train.shape[0]
                seg_list_idx = seg_list_idx + 1
            data_train, data_num_train, label_train = \
                data_utils.grouped_shuffle([data_train, data_num_train, label_train])

        offset = int(random.gauss(0, sample_num * setting.sample_num_variance))
        offset = max(offset, -sample_num * setting.sample_num_clip)
        offset = min(offset, sample_num * setting.sample_num_clip)
        sample_num_train = sample_num + offset
        xforms_np, rotations_np = pf.get_xforms(batch_size_train,
                                                rotation_range=rotation_range,
                                                scaling_range=scaling_range,
                                                order=setting.rotation_order)
        sess.run(reset_metrics_op)
        sess.run([train_op, loss_mean_update_op, t_1_acc_update_op, t_1_per_class_acc_update_op],
                 feed_dict={
                     pts_fts: points_batch,
                     indices: pf.get_indices(batch_size_train, sample_num_train, points_num_batch),
                     xforms: xforms_np,
                     rotations: rotations_np,
                     jitter_range: np.array([jitter]),
                     labels_seg: labels_batch,
                     labels_weights: weights_batch,
                     is_training: True,
                 })
        if batch_idx_train % 10 == 0:
            loss, t_1_acc, t_1_per_class_acc, summaries, step = sess.run([loss_mean_op,
                                                                    t_1_acc_op,
                                                                    t_1_per_class_acc_op,
                                                                    summaries_op,
                                                                    global_step])
            summary_writer.add_summary(summaries, step)
            print('{}-[Train]-Iter: {:06d}  Loss: {:.4f}  T-1 Acc: {:.4f}  T-1 mAcc: {:.4f}'
                  .format(datetime.now(), step, loss, t_1_acc, t_1_per_class_acc))
            sys.stdout.flush()
        ######################################################################

        if batch_idx_train % 500 == 0:
            filename_ckpt = os.path.join(folder_ckpt, 'iter')
            saver.save(sess, filename_ckpt, global_step=global_step)
            print('{}-Checkpoint saved to {}!'.format(datetime.now(), filename_ckpt))        
        
        
        ######################################################################
        # Validation
        if (batch_idx_train % step_val == 0 and (batch_idx_train != 0 or args.load_ckpt is not None)) \
                or batch_idx_train == batch_num - 1:
            filename_ckpt = os.path.join(folder_ckpt, 'iter')
            saver.save(sess, filename_ckpt, global_step=global_step)
            print('{}-Checkpoint saved to {}!'.format(datetime.now(), filename_ckpt))

            sess.run(reset_metrics_op)
            for batch_val_idx in range(batch_num_val):
                start_idx = batch_size * batch_val_idx
                end_idx = min(start_idx + batch_size, num_val)
                batch_size_val = end_idx - start_idx
                points_batch = data_val[start_idx:end_idx, ...]
                points_num_batch = data_num_val[start_idx:end_idx, ...]
                labels_batch = label_val[start_idx:end_idx, ...]
                
                weights_batch = np.array(label_weights_list)[labels_batch]

                xforms_np, rotations_np = pf.get_xforms(batch_size_val,
                                                        rotation_range=rotation_range_val,
                                                        scaling_range=scaling_range_val,
                                                        order=setting.rotation_order)
                sess.run([loss_mean_update_op, t_1_acc_update_op, t_1_per_class_acc_update_op],
                         feed_dict={
                             pts_fts: points_batch,
                             indices: pf.get_indices(batch_size_val, sample_num, points_num_batch),
                             xforms: xforms_np,
                             rotations: rotations_np,
                             jitter_range: np.array([jitter_val]),
                             labels_seg: labels_batch,
                             labels_weights: weights_batch,
                             is_training: False,
                         })
            loss_val, t_1_acc_val, t_1_per_class_acc_val, summaries_val, step = sess.run(
                [loss_mean_op, t_1_acc_op, t_1_per_class_acc_op, summaries_val_op, global_step])
            summary_writer.add_summary(summaries_val, step)
            print('{}-[ Val ]-Average:      Loss: {:.4f}  T-1 Acc: {:.4f}  T-1 mAcc: {:.4f}'
                  .format(datetime.now(), loss_val, t_1_acc_val, t_1_per_class_acc_val))
            sys.stdout.flush()
        ######################################################################
    print('{}-Done!'.format(datetime.now()))

2019-12-30 13:10:28.018155-Found checkpoint /home/kartik/DL_model/PointCNN/outfolder_argo_gr/ckpts/iter-8003
INFO:tensorflow:Restoring parameters from /home/kartik/DL_model/PointCNN/outfolder_argo_gr/ckpts/iter-8003
2019-12-30 13:10:28.444912-Checkpoint loaded from /home/kartik/DL_model/PointCNN/outfolder_argo_gr/ckpts/iter-8003 (Iter 8003)
2019-12-30 13:10:37.781530-[Train]-Iter: 008004  Loss: 0.0823  T-1 Acc: 0.9431  T-1 mAcc: 0.1174
INFO:tensorflow:/home/kartik/DL_model/PointCNN/outfolder_argo_gr/ckpts/iter-8004 is not in all_model_checkpoint_paths. Manually adding it.
2019-12-30 13:10:39.054225-Checkpoint saved to /home/kartik/DL_model/PointCNN/outfolder_argo_gr/ckpts/iter!
2019-12-30 13:10:39.817141-[Train]-Iter: 008014  Loss: 0.2481  T-1 Acc: 0.4807  T-1 mAcc: 0.0567
2019-12-30 13:10:40.578258-[Train]-Iter: 008024  Loss: 0.0348  T-1 Acc: 0.9975  T-1 mAcc: 0.1245
2019-12-30 13:10:41.338301-[Train]-Iter: 008034  Loss: 0.0612  T-1 Acc: 0.9989  T-1 mAcc: 0.1249
2019-12-30 13:10:42.09

2019-12-30 13:11:38.087106-[Train]-Iter: 008764  Loss: 0.6461  T-1 Acc: 0.2398  T-1 mAcc: 0.0625
2019-12-30 13:11:38.859308-[Train]-Iter: 008774  Loss: 0.0316  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:11:39.631774-[Train]-Iter: 008784  Loss: 0.0472  T-1 Acc: 0.9992  T-1 mAcc: 0.1248
2019-12-30 13:11:40.403333-[Train]-Iter: 008794  Loss: 0.0135  T-1 Acc: 0.9954  T-1 mAcc: 0.1245
2019-12-30 13:11:41.174999-[Train]-Iter: 008804  Loss: 0.3104  T-1 Acc: 0.4662  T-1 mAcc: 0.0624
2019-12-30 13:11:41.947300-[Train]-Iter: 008814  Loss: 0.0987  T-1 Acc: 0.9967  T-1 mAcc: 0.1247
2019-12-30 13:11:42.717930-[Train]-Iter: 008824  Loss: 0.0628  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:11:43.488901-[Train]-Iter: 008834  Loss: 0.0192  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:11:44.258837-[Train]-Iter: 008844  Loss: 0.0176  T-1 Acc: 0.9988  T-1 mAcc: 0.1249
2019-12-30 13:11:45.029316-[Train]-Iter: 008854  Loss: 0.0123  T-1 Acc: 0.9980  T-1 mAcc: 0.1248
2019-12-30 13:11:45.800790-[Tr

2019-12-30 13:13:47.723679-[Train]-Iter: 009524  Loss: 0.2763  T-1 Acc: 0.5105  T-1 mAcc: 0.0625
2019-12-30 13:13:48.496239-[Train]-Iter: 009534  Loss: 0.0818  T-1 Acc: 0.9996  T-1 mAcc: 0.1249
2019-12-30 13:13:49.271125-[Train]-Iter: 009544  Loss: 0.1621  T-1 Acc: 0.9999  T-1 mAcc: 0.1250
2019-12-30 13:13:50.045113-[Train]-Iter: 009554  Loss: 0.0389  T-1 Acc: 0.9997  T-1 mAcc: 0.1250
2019-12-30 13:13:50.819893-[Train]-Iter: 009564  Loss: 0.0988  T-1 Acc: 0.9902  T-1 mAcc: 0.1241
2019-12-30 13:13:51.592837-[Train]-Iter: 009574  Loss: 0.0025  T-1 Acc: 0.9998  T-1 mAcc: 0.1250
2019-12-30 13:13:52.367059-[Train]-Iter: 009584  Loss: 0.3905  T-1 Acc: 0.3185  T-1 mAcc: 0.0625
2019-12-30 13:13:53.142274-[Train]-Iter: 009594  Loss: 0.1667  T-1 Acc: 0.9065  T-1 mAcc: 0.1168
2019-12-30 13:13:53.916782-[Train]-Iter: 009604  Loss: 0.0484  T-1 Acc: 0.9822  T-1 mAcc: 0.1232
2019-12-30 13:13:54.689848-[Train]-Iter: 009614  Loss: 0.1519  T-1 Acc: 0.5603  T-1 mAcc: 0.0625
2019-12-30 13:13:55.463693-[Tr

2019-12-30 13:15:57.698461-[Train]-Iter: 010314  Loss: 0.0784  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:15:58.472498-[Train]-Iter: 010324  Loss: 0.1145  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:15:59.247577-[Train]-Iter: 010334  Loss: 0.0712  T-1 Acc: 0.8743  T-1 mAcc: 0.1094
2019-12-30 13:16:00.019371-[Train]-Iter: 010344  Loss: 0.0016  T-1 Acc: 0.9998  T-1 mAcc: 0.1250
2019-12-30 13:16:00.793227-[Train]-Iter: 010354  Loss: 0.1049  T-1 Acc: 0.9984  T-1 mAcc: 0.1248
2019-12-30 13:16:01.564248-[Train]-Iter: 010364  Loss: 0.0102  T-1 Acc: 0.9993  T-1 mAcc: 0.1249
2019-12-30 13:16:02.337510-[Train]-Iter: 010374  Loss: 0.2420  T-1 Acc: 0.3587  T-1 mAcc: 0.0647
2019-12-30 13:16:03.110942-[Train]-Iter: 010384  Loss: 0.0204  T-1 Acc: 0.9995  T-1 mAcc: 0.1249
2019-12-30 13:16:03.884411-[Train]-Iter: 010394  Loss: 0.0273  T-1 Acc: 0.9997  T-1 mAcc: 0.1250
2019-12-30 13:16:04.656812-[Train]-Iter: 010404  Loss: 0.7756  T-1 Acc: 0.2334  T-1 mAcc: 0.0625
2019-12-30 13:16:05.429843-[Tr

2019-12-30 13:18:05.597905-[Train]-Iter: 011074  Loss: 0.0307  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:18:06.370115-[Train]-Iter: 011084  Loss: 0.0319  T-1 Acc: 0.9996  T-1 mAcc: 0.1249
2019-12-30 13:18:07.143354-[Train]-Iter: 011094  Loss: 0.0348  T-1 Acc: 0.9994  T-1 mAcc: 0.1249
2019-12-30 13:18:07.918567-[Train]-Iter: 011104  Loss: 0.0024  T-1 Acc: 0.9994  T-1 mAcc: 0.1249
2019-12-30 13:18:08.692995-[Train]-Iter: 011114  Loss: 0.3690  T-1 Acc: 0.2345  T-1 mAcc: 0.0625
2019-12-30 13:18:09.466617-[Train]-Iter: 011124  Loss: 0.0335  T-1 Acc: 0.9990  T-1 mAcc: 0.1249
2019-12-30 13:18:10.240585-[Train]-Iter: 011134  Loss: 0.0177  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:18:11.013170-[Train]-Iter: 011144  Loss: 0.0083  T-1 Acc: 0.9984  T-1 mAcc: 0.1248
2019-12-30 13:18:11.786383-[Train]-Iter: 011154  Loss: 0.0819  T-1 Acc: 0.6515  T-1 mAcc: 0.0625
2019-12-30 13:18:12.559299-[Train]-Iter: 011164  Loss: 0.0796  T-1 Acc: 0.6581  T-1 mAcc: 0.0625
2019-12-30 13:18:13.332572-[Tr

...

2019-12-30 13:40:41.529907-[Train]-Iter: 020474  Loss: 0.0072  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:40:42.302111-[Train]-Iter: 020484  Loss: 0.0012  T-1 Acc: 0.9997  T-1 mAcc: 0.1250
2019-12-30 13:40:43.076803-[Train]-Iter: 020494  Loss: 0.0027  T-1 Acc: 0.9969  T-1 mAcc: 0.1245
2019-12-30 13:40:43.851805-[Train]-Iter: 020504  Loss: 0.0290  T-1 Acc: 0.9998  T-1 mAcc: 0.1250
INFO:tensorflow:/home/kartik/DL_model/PointCNN/outfolder_argo_gr/ckpts/iter-20504 is not in all_model_checkpoint_paths. Manually adding it.
2019-12-30 13:40:44.678217-Checkpoint saved to /home/kartik/DL_model/PointCNN/outfolder_argo_gr/ckpts/iter!
2019-12-30 13:40:45.452627-[Train]-Iter: 020514  Loss: 0.0727  T-1 Acc: 0.9440  T-1 mAcc: 0.1197
2019-12-30 13:40:46.227888-[Train]-Iter: 020524  Loss: 0.0906  T-1 Acc: 0.6037  T-1 mAcc: 0.0782
2019-12-30 13:40:47.000693-[Train]-Iter: 020534  Loss: 0.1720  T-1 Acc: 0.9861  T-1 mAcc: 0.1240
2019-12-30 13:40:47.774244-[Train]-Iter: 020544  Loss: 0.3260  T-1 Acc: 0

2019-12-30 13:42:51.017454-[Train]-Iter: 021234  Loss: 0.0264  T-1 Acc: 0.9998  T-1 mAcc: 0.1250
2019-12-30 13:42:51.788801-[Train]-Iter: 021244  Loss: 0.0003  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:42:52.561820-[Train]-Iter: 021254  Loss: 0.0012  T-1 Acc: 0.9984  T-1 mAcc: 0.1248
2019-12-30 13:42:53.334932-[Train]-Iter: 021264  Loss: 0.0001  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:42:54.110058-[Train]-Iter: 021274  Loss: 0.3820  T-1 Acc: 0.3267  T-1 mAcc: 0.0625
2019-12-30 13:42:54.883999-[Train]-Iter: 021284  Loss: 0.0429  T-1 Acc: 0.9978  T-1 mAcc: 0.1248
2019-12-30 13:42:55.657737-[Train]-Iter: 021294  Loss: 0.2150  T-1 Acc: 0.7929  T-1 mAcc: 0.1087
2019-12-30 13:42:56.431993-[Train]-Iter: 021304  Loss: 0.2365  T-1 Acc: 0.4538  T-1 mAcc: 0.0625
2019-12-30 13:42:57.205847-[Train]-Iter: 021314  Loss: 0.1084  T-1 Acc: 0.5213  T-1 mAcc: 0.0706
2019-12-30 13:42:57.979632-[Train]-Iter: 021324  Loss: 0.0779  T-1 Acc: 0.6206  T-1 mAcc: 0.0625
2019-12-30 13:42:58.757920-[Tr

2019-12-30 13:43:54.058789-Checkpoint saved to /home/kartik/DL_model/PointCNN/outfolder_argo_gr/ckpts/iter!
2019-12-30 13:45:01.215971-[ Val ]-Average:      Loss: 0.1014  T-1 Acc: 0.9578  T-1 mAcc: 0.2026
2019-12-30 13:45:01.995581-[Train]-Iter: 022014  Loss: 0.0962  T-1 Acc: 0.9482  T-1 mAcc: 0.1199
2019-12-30 13:45:02.770254-[Train]-Iter: 022024  Loss: 0.0256  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:45:03.543860-[Train]-Iter: 022034  Loss: 0.0162  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:45:04.317302-[Train]-Iter: 022044  Loss: 0.2713  T-1 Acc: 0.7761  T-1 mAcc: 0.1088
2019-12-30 13:45:05.090805-[Train]-Iter: 022054  Loss: 0.0935  T-1 Acc: 0.9796  T-1 mAcc: 0.1230
2019-12-30 13:45:05.864555-[Train]-Iter: 022064  Loss: 0.1293  T-1 Acc: 0.9236  T-1 mAcc: 0.1183
2019-12-30 13:45:06.637024-[Train]-Iter: 022074  Loss: 0.4579  T-1 Acc: 0.1663  T-1 mAcc: 0.0638
2019-12-30 13:45:07.410744-[Train]-Iter: 022084  Loss: 0.0100  T-1 Acc: 0.9731  T-1 mAcc: 0.1171
2019-12-30 13:45:08

2019-12-30 13:46:04.768108-[Train]-Iter: 022814  Loss: 0.0022  T-1 Acc: 0.9998  T-1 mAcc: 0.1250
2019-12-30 13:46:05.545856-[Train]-Iter: 022824  Loss: 0.0025  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:46:06.323476-[Train]-Iter: 022834  Loss: 0.0006  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:46:07.100881-[Train]-Iter: 022844  Loss: 0.0024  T-1 Acc: 0.9956  T-1 mAcc: 0.1245
2019-12-30 13:46:07.879958-[Train]-Iter: 022854  Loss: 0.0175  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:46:08.659572-[Train]-Iter: 022864  Loss: 0.0015  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:46:09.436610-[Train]-Iter: 022874  Loss: 0.0532  T-1 Acc: 0.9616  T-1 mAcc: 0.1208
2019-12-30 13:46:10.215746-[Train]-Iter: 022884  Loss: 0.0002  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:46:10.995473-[Train]-Iter: 022894  Loss: 0.0137  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:46:11.771876-[Train]-Iter: 022904  Loss: 0.0024  T-1 Acc: 0.9998  T-1 mAcc: 0.1250
2019-12-30 13:46:12.550035-[Tr

2019-12-30 13:48:13.749493-[Train]-Iter: 023574  Loss: 0.0003  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:48:14.523436-[Train]-Iter: 023584  Loss: 0.0080  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:48:15.297380-[Train]-Iter: 023594  Loss: 0.0003  T-1 Acc: 1.0000  T-1 mAcc: 0.1250
2019-12-30 13:48:16.071684-[Train]-Iter: 023604  Loss: 0.0484  T-1 Acc: 0.9979  T-1 mAcc: 0.1248


In [18]:
filename_ckpt

'/home/kartik/DL_model/PointCNN/outfolder_argo/ckpts/iter'